#### imports

In [0]:
import torch
import os
from  PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [0]:
image_dir ='/Workspace/sid-v2/computervision1/Classification_dataset_v3/images/train'
for label, class_dir in enumerate(os.listdir(image_dir)):
    print(label,class_dir)

In [0]:
class generate_image_dataset(Dataset):
  def __init__(self,image_dir, transform=None):
    self.image_dir=image_dir
    self.image_paths=[]
    self.labels=[]
    self.class_name={}
    self.transform = transform

    for label, class_dir in enumerate(os.listdir(image_dir)):
        self.class_name[label] =class_dir
        class_path = os.path.join(image_dir,class_dir)
        for img_name in os.listdir(class_path):
          self.image_paths.append(os.path.join(class_path,img_name))
          self.labels.append(label)

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self,idx):
    image_path = self.image_paths[idx]
    image = Image.open(image_path).convert('RGB')     
    label = self.labels[idx]             

    if self.transform:
        image = self.transform(image)     

    return image, label

In [0]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5,0.5,0.5])]
    )


train_image_dir ='/Workspace/sid-v2/computervision1/Classification_dataset_v3/images/train'
test_image_dir ='/Workspace/sid-v2/computervision1/Classification_dataset_v3/images/test'

training_image_dataset = generate_image_dataset(image_dir=train_image_dir,transform=transform)
test_image_dataset = generate_image_dataset(image_dir=test_image_dir,transform=transform)

train_image_loader = DataLoader(dataset=training_image_dataset, batch_size=32,shuffle=True)
test_image_loader = DataLoader(dataset=test_image_dataset, batch_size=32,shuffle=True)

In [0]:
for images, labels in train_image_loader:
    print(images.shape, labels.shape)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

for images, labels in train_image_loader:
    print(images.shape, labels.shape)
    img =  images[0].numpy()
    label = labels[0].item()

    print(training_image_dataset.class_name[label])
    img = np.transpose(img, (1, 2, 0))
    print(img.shape)
    print(label)

    plt.imshow(img,vmin=-1,vmax=1)
    break
    

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

class customcnnmodel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(customcnnmodel,self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes #3 predictions
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),    
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),    
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),    
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.to_linear = None
        self.get_conv_output(self.input_dim)
        
        self.fc_layers = nn.Sequential(
            nn.Linear(self._to_linear, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def get_conv_output(self, input_dim = 128 ):
        with torch.no_grad():
            dummyinput = torch.zeros(1,3, input_dim, input_dim)
            output = self.conv_layers(dummyinput)
            self._to_linear = output.view(output.size(0), -1).size(1)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

device =  torch.device("cpu")

model = customcnnmodel(input_dim=128, num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 2

for i in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_image_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {i+1}/{epochs}, Average Loss: {running_loss/len(train_image_loader):.4f}")

print("Training completed!")